In [7]:
# Import the dependencies.
import pandas as pd
import gmaps
import requests
# Import the API key.
from config import g_key

In [8]:
# Store the CSV you saved created in part one into a DataFrame.
city_data_df = pd.read_csv("weather_data/cities.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Ketchikan,US,2022-02-15 22:43:43,55.3422,-131.6461,40.98,98,100,2.10
1,1,Kaitangata,NZ,2022-02-15 22:43:43,-46.2817,169.8464,62.92,63,86,7.43
2,2,Batticaloa,LK,2022-02-15 22:43:43,7.7102,81.6924,75.15,73,70,8.10
3,3,Yellowknife,CA,2022-02-15 22:42:45,62.4560,-114.3525,-16.58,70,0,9.22
4,4,Hermanus,ZA,2022-02-15 22:43:44,-34.4187,19.2345,68.81,78,73,9.64


In [9]:
city_data_df.dtypes

City_ID         int64
City           object
Country        object
Date           object
Lat           float64
Lng           float64
Max Temp      float64
Humidity        int64
Cloudiness      int64
Wind Speed    float64
dtype: object

In [10]:
# Configure gmaps to use your Google API key.
gmaps.configure(api_key=g_key)

In [11]:
# Heatmap of temperature
# Get the latitude and longitude.
locations = city_data_df[["Lat", "Lng"]]
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
# Assign the figure variable.
fig = gmaps.figure()
# Assign the heatmap variable.
heat_layer = heat_layer = gmaps.heatmap_layer(locations, weights=[max(temp, 0) for temp in max_temp], dissipating=False, max_intensity=300, point_radius=4)
# Add the heatmap layer.
fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [12]:
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)

In [13]:
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [14]:
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)


,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
2,2,Batticaloa,LK,2022-02-15 22:43:43,7.7102,81.6924,75.15,73,70,8.10
5,5,Bongaree,AU,2022-02-15 22:43:44,-27.0797,153.1586,79.29,64,21,15.79
6,6,Karratha,AU,2022-02-15 22:43:44,-20.7377,116.8463,86.90,66,4,9.75
8,8,Soyo,AO,2022-02-15 22:43:45,-6.1349,12.3689,79.25,89,100,5.26
10,10,Buchanan,LR,2022-02-15 22:43:46,5.8808,-10.0467,78.76,87,71,6.22
14,14,Avarua,CK,2022-02-15 22:43:47,-21.2078,-159.7750,80.65,78,40,10.36
20,20,Port Blair,IN,2022-02-15 22:43:49,11.6667,92.7500,78.84,94,75,4.61
21,21,Beira,MZ,2022-02-15 22:43:50,-19.8436,34.8389,82.49,78,75,14.97
27,27,Rikitea,PF,2022-02-15 22:41:47,-23.1203,-134.9692,78.60,66,50,10.00
28,28,Kapaa,US,2022-02-15 22:43:52,22.0752,-159.3190,79.20,65,20,13.80


In [15]:
preferred_cities_df.count()

City_ID       168
City          168
Country       168
Date          168
Lat           168
Lng           168
Max Temp      168
Humidity      168
Cloudiness    168
Wind Speed    168
dtype: int64

In [16]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
2,Batticaloa,LK,75.15,7.7102,81.6924,
5,Bongaree,AU,79.29,-27.0797,153.1586,
6,Karratha,AU,86.90,-20.7377,116.8463,
8,Soyo,AO,79.25,-6.1349,12.3689,
10,Buchanan,LR,78.76,5.8808,-10.0467,
14,Avarua,CK,80.65,-21.2078,-159.7750,
20,Port Blair,IN,78.84,11.6667,92.7500,
21,Beira,MZ,82.49,-19.8436,34.8389,
27,Rikitea,PF,78.60,-23.1203,-134.9692,
28,Kapaa,US,79.20,22.0752,-159.3190,


In [17]:
from config import g_key

In [27]:
# Set the parameters to search for a hotel in Paris.
params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key,
    "location": "48.8566, 2.3522"}

# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

# Add the latitude and longitude to location key for the params dictionary.
params["location"] = f"{lat},{lng}"

# Use the search term: "lodging" and our latitude and longitude.
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# Make request and get the JSON data from the search.
hotels = requests.get(base_url, params=params).json()
    
# Grab the first hotel from the results and store the name.
try:
    hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
except (IndexError):
    print("Hotel not found... skipping.")
    
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
2,Batticaloa,LK,75.15,7.7102,81.6924,
5,Bongaree,AU,79.29,-27.0797,153.1586,
6,Karratha,AU,86.90,-20.7377,116.8463,
8,Soyo,AO,79.25,-6.1349,12.3689,
10,Buchanan,LR,78.76,5.8808,-10.0467,
14,Avarua,CK,80.65,-21.2078,-159.7750,
20,Port Blair,IN,78.84,11.6667,92.7500,
21,Beira,MZ,82.49,-19.8436,34.8389,
27,Rikitea,PF,78.60,-23.1203,-134.9692,
28,Kapaa,US,79.20,22.0752,-159.3190,


In [29]:
# Add a heatmap of temperature for the vacation spots and marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,
             dissipating=False, max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [31]:
# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

In [32]:
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))